In [27]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [24]:
from keras.preprocessing.image import load_img, img_to_array
files = os.listdir(r'.\AnimeFaceDetecter\Datas\result\face')
images = []
for i in files:
    image = load_img(r'.\AnimeFaceDetecter\Datas\result\face\{}'.format(i),target_size=(64,64))
    image = img_to_array(image)
    image.shape
    image = image / 256
    images.append(image)

In [25]:
np.array(images).shape

(654, 64, 64, 3)

In [26]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size = [batch_size, n_noise])

In [29]:
def windowUp(arr):
    frame = cv2.cvtColor(np.array(arr), cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [30]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 1])
Z = tf.placeholder(tf.float32, [None, 100])

In [31]:
def generate(Z):
    w1 = tf.Variable(tf.random_normal([100, 16384]))
    h1 = tf.matmul(Z,w1)
    re = tf.reshape(h1,[-1,4,4,1024])
    re = tf.layers.batch_normalization(re)

    conv_h1 = tf.layers.conv2d_transpose(re ,512, 5, 2, padding='same')
    
    conv_h1 = tf.nn.relu(conv_h1)

    conv_h2 = tf.layers.conv2d_transpose(conv_h1 ,256, 4, 2, padding='same')
    conv_h2 = tf.nn.relu(conv_h2)

    conv_h3 = tf.layers.conv2d_transpose(conv_h2 ,128, 4, 2, padding='same')
    conv_h3 = tf.nn.relu(conv_h3)

    conv_h5 = tf.layers.conv2d_transpose(conv_h3 ,3, 4, 2, padding='same')
    return conv_h5

In [32]:
def discriminaster(inputs):
    W1 = tf.Variable(tf.random_normal([3,3,3,32],stddev=0.02))
    H1 = tf.nn.conv2d(inputs,W1, strides=[1,2,2,1], padding="SAME")
    H1 = tf.layers.batch_normalization(H1)
    H1 = tf.nn.relu(H1)

    W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.02))
    H2 = tf.nn.conv2d(H1,W2, strides=[1,2,2,1], padding="SAME")
    H2 = tf.nn.relu(H2)

    W3 = tf.Variable(tf.random_normal([3,3,64,256],stddev=0.02))
    H3 = tf.nn.conv2d(H2,W3, strides=[1,2,2,1], padding="SAME")
    H3 = tf.nn.relu(H3)
    
    W4 = tf.Variable(tf.random_normal([3,3,256,1024],stddev=0.02))
    H4 = tf.nn.conv2d(H3,W4, strides=[1,2,2,1], padding="SAME")
    H4 = tf.nn.relu(H4)
    
    out = tf.reshape(H4,[-1,16384])
    out = tf.layers.dense(out,256)
    out = tf.layers.dense(out,1)
    return out

In [33]:

G = generate(Z)
D_real = discriminaster(X)
D_gene = discriminaster(G)

In [35]:
loss_D_real = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_real,
        labels=tf.ones_like(D_real)
    )
)

loss_D_gene = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=D_gene,
        labels=tf.zeros_like(D_gene)
    )
)

In [36]:
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [37]:
train_D = tf.train.AdamOptimizer(0.001).minimize(loss_D)
train_G = tf.train.AdamOptimizer(0.001).minimize(loss_G)

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [48]:
for i in range(10):
    noise = get_noise(1,100)
    _, varsD=sess.run([train_D, loss_D], 
                  feed_dict={
                      X:np.array(images),
                      Z: noise})
    _, varsG=sess.run([train_G, loss_G], 
                  feed_dict={
                      X:np.array(images),
                      Z: noise})
    print(str(i)+"   D : "+str(varsD)+"\t G : "+str(varsG))

0   D : 0.17413387	 G : 11.1477995
1   D : 0.18718398	 G : 11.238913
2   D : 0.21939513	 G : 11.3199415
3   D : 0.0912514	 G : 14.570533
4   D : 0.22721289	 G : 11.152951
5   D : 0.50218296	 G : 5.894978
6   D : 0.13318141	 G : 13.414516
7   D : 0.009151893	 G : 18.493025
8   D : 0.2053367	 G : 11.729457
9   D : 0.091280535	 G : 12.535742


In [49]:
noise = get_noise(30,100)
img = sess.run(G, feed_dict={Z:noise})
for i in range(5):
    windowUp(img[i])

In [44]:
windowUp(images[3])